### Can we classify each phase as relatively low or high anxiety for each subject? ###

In [119]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_validate, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr.Phases.PHASES_LIST, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 40].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

In [120]:
importlib.reload(train)

models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.01, 0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
        "probability": [True]
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50, 60],
        "max_depth": [3, 4, 5, 6, 7, 8],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5],
        "random_state": [39]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.001, 0.01, 0.1, 0.2, 0.3],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40, 50],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [39]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
# ]
] + train.Metrics.STATISTICAL

In [122]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)


random.seed(39)

model_phases = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

threshold = "fixed"

anxiety_label_type = None
x, y = train.Train_APD.get_apd_data_ranking(
    metrics, model_phases, verbose=False, anxiety_label_type=anxiety_label_type, 
    threshold=threshold, normalize=True, combine_phases=False, standardize=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

# drop subjects with noisy data
# x = x[~x["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
# y = y[~y["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

if anxiety_label_type is not None:
    x.drop(labels=["anxietyGroup"], axis=1)
    
acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}
ensemble_weights = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.1, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    features = {name: metrics+["lf_hf_ratio"] for name in models.keys()}
    # features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        # if get_importance:
            # try:
                # print("")
                # feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                # feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                # print(feature_imp)
            # except Exception as e:
                # print(out[model_name]["performance"][2])
            # print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        ensemble_weights[model_name] = acc
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.54241696 0.56499038 0.55831426 0.55643473 0.54147644 0.54805493
 0.56698643 0.5515663  0.54197246 0.55815681 0.55077985 0.55712843
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 'scale', 'kernel': 'poly', 'probability': True}
LGB: {'max_depth': 8, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 30, 'random_state': 39}
XGB: {'eval_metric': 'error', 'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 40, 'objective': 'binary:logistic', 'random_state': 39, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [66 50], Predictions: [0 1], [115   1]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [66 50], Predictions: [0 1], [79 37]
Training RF ...
Model RF, Actual: [0 1], [66 50], Predictions: [0 1], [95 21]
Training XGB ...
Model XGB, Actual: [0 1], [66 50], Predictions: [0 1], [111   5]
Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.56117357 0.56372455 0.58258294 0.58714145 0.56271676 0.56501706
 0.57091216 0.57132731 0.54360226 0.56744482 0.56626387 0.57180903
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 1, 'kernel': 'poly', 'probability': True}
LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 30, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 40, 'random_state': 39}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 39, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [51 65], Predictions: [0 1], [115   1]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [51 65], Predictions: [0 1], [89 27]
Training RF ...
Model RF, Actual: [0 1], [51 65], Predictions: [0 1], [103  13]
Training XGB ...
Model XGB, Actual: [0 1], [51 65], Predictions: [0 1], [102  14]
Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


...

Grid search for XGB ...
SVM: {'C': 0.1, 'gamma': 'scale', 'kernel': 'poly', 'probability': True}
LGB: {'max_depth': 8, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 20, 'random_state': 39}
XGB: {'eval_metric': 'error', 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 40, 'objective': 'binary:logistic', 'random_state': 39, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [55 63], Predictions: [0 1], [115   3]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [55 63], Predictions: [0 1], [92 26]
Training RF ...
Model RF, Actual: [0 1], [55 63], Predictions: [0 1], [106  12]
Training XGB ...
Model XGB, Actual: [0 1], [55 63], Predictions: [0 1], [101  17]
Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.57279524 0.56614183 0.57895685 0.57246053 0.56488562 0.56540908
 0.57820559 0.5772339  0.55575715 0.54091509 0.54540341 0.55759995
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 30, 'random_state': 39}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 39, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [50 76], Predictions: [0 1], [111  15]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [50 76], Predictions: [0 1], [95 31]
Training RF ...
Model RF, Actual: [0 1], [50 76], Predictions: [0 1], [97 29]
Training XGB ...
Model XGB, Actual: [0 1], [50 76], Predictions: [0 1], [100  26]
Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.55520069 0.5475004  0.55577436 0.55901934 0.56234486 0.55328192
 0.55468318 0.56649566 0.56401516 0.56250613 0.56068855 0.57077406
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 40, 'random_state': 39}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 39, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [76 41], Predictions: [0 1], [95 22]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [76 41], Predictions: [0 1], [89 28]
Training RF ...
Model RF, Actual: [0 1], [76 41], Predictions: [0 1], [84 33]
Training XGB ...
Model XGB, Actual: [0 1], [76 41], Predictions: [0 1], [105  12]
Model evaluation metrics for SVM:
	Accuracy: 0.5603448275862069
	Precision: 0.0
	Recall: 0.0
	F1-score: 0.0
	AUC score: 0.49242424242424243
----------------------------------------
	A

In [167]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)

from scipy.stats import mode
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


random.seed(39)

def train_predict_ensemble(ensemble_models, x_train, y_train, x_test, y_test, features, type="majority_vote", weights=None):
    y_preds = []
    if type == "majority_vote":
        # features = list(features.values())[0]
        # ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
        # ensemble = VotingClassifier(estimators=ensemble_models, voting='hard', weights=weights)
        # ensemble.fit(x_train.loc[:, features], y_train)
        # y_preds = ensemble.predict(x_test.loc[:, features])
        for model_name in ensemble_models:
            x_test_temp = x_test.loc[:, features[model_name]]
            y_pred = ensemble_models[model_name].predict(x_test_temp)
            y_preds.append(y_pred)
        y_preds = mode(y_preds, axis=0)[0]
        y_preds = np.reshape(y_preds, (y_preds.shape[1], 1))
    elif type == "weighted_avg":
        features = list(features.values())[0]
        ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
        ensemble = VotingClassifier(estimators=ensemble_models, voting='soft', weights=weights)
        ensemble.fit(x_train.loc[:, features], y_train)
        y_preds = ensemble.predict(x_test.loc[:, features])

    return y_preds


test_size = 0.1
folds = 5
ensemble_models = [
    "SVM", 
    "LGB",
    "RF",
    "XGB"
]
voting_type = "weighted_avg"

cv, x_train, y_train, x_test, y_test = train.kfold_train_test_split(x, y, test_size, is_resample=False, folds=folds)
y_train = y_train.loc[:, "label"]
y_test = y_test.loc[:, "label"]

estimators = {name: best_models[name] for name in ensemble_models}
weights = [ensemble_weights[model_name] for model_name in estimators.keys()]
# weights = np.divide(weights, np.sum(weights))
# weights = np.divide(weights, len(weights))
weights = [1 for _ in range(len(list(estimators.keys())))]
y_pred = train_predict_ensemble(estimators, x_train, y_train, x_test, y_test, features, type=voting_type, weights=weights)

acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, zero_division=0)
r = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred)
print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)

	Accuracy: 0.6779661016949152
	Precision: 0.8333333333333334
	Recall: 0.37037037037037035
	F1-score: 0.5128205128205128
	AUC score: 0.6539351851851851
----------------------------------------
